# 집계 연산
- 빅데이터를 사용해 연산을 수행하는 경우 질문에 대한 정확한 답을 얻기 위해서는 연산, 네트워크, 저장소 등 상당한 비용이 들 수밖에 없다.
- 따라서 수용 가능한 정도의 정확도에 맞춰 <strong>근사치</strong>를 계산하는 것이 비용을 고려했을 때 효율적

In [0]:
from pyspark.sql import functions as F

In [0]:
path='/FileStore/tables/all/*.csv'

In [0]:
df = spark.read.format('csv').option('header','true').option('inferSchema', 'true').load(path).coalesce(5)

In [0]:
df.cache()
df.createOrReplaceTempView('dfTable')

In [0]:
df.count()

Out[4]: 541909

---
- count 메서드는 트랜스포메이션이 아닌 액션이라 전체 크기를 알아보는 용도가 아닌 캐싱 작업을 수행하는 용도로 사용되기도 한다.

## 집계 함수

### count

In [0]:
#다음 count 함수는 액션이 아닌 트랜스포메이션
df.select(F.count('StockCode')).show()

+----------------+
count(StockCode)|
+----------------+
 541909|
+----------------+

----
- count(*) 구문은 null값을 가진 로우를 포함
- count함수에 특정 컬럼을 지정하면 null값 포함 X

### countDistinct

In [0]:
#전체 레코드 수가 아닌 고유 레코드 수를 카운트
df.select(F.countDistinct('StockCode')).show()

+-------------------------+
count(DISTINCT StockCode)|
+-------------------------+
 4070|
+-------------------------+

### approx_count_distinct

In [0]:
#근사치만으로도 유의미하다면 해당 함수를 이용해 근사치 계산
df.select(F.approx_count_distinct('StockCode',0.1)).show()

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3364|
+--------------------------------+

----
- 0.1은 최대 추정 오류율(maximum estimation error)
- 위 예제에선 큰 오류율을 설정했기에 기대치에서 크게 벗어나는 결과를 얻었지만, countDistinct보다 빠르게 결과 반환
  - 대규모 데이터셋을 사용할 때 훨씬 더 성능이 좋아짐

### first와 last

In [0]:
#row 기반 동작
df.select(F.first('StockCode'), F.last('StockCode')).show()

+----------------+---------------+
first(StockCode)|last(StockCode)|
+----------------+---------------+
 85123A| 22138|
+----------------+---------------+

### min과 max

In [0]:
df.select(F.min('Quantity'), F.max('Quantity')).show()

+-------------+-------------+
min(Quantity)|max(Quantity)|
+-------------+-------------+
 -80995| 80995|
+-------------+-------------+

### sum

In [0]:
df.select(F.sum('Quantity')).show()

+-------------+
sum(Quantity)|
+-------------+
 5176450|
+-------------+

### sumDistinct

In [0]:
#고윳값 합산
df.select(F.sumDistinct('Quantity')).show()

+----------------------+
sum(DISTINCT Quantity)|
+----------------------+
 29310|
+----------------------+

### avg

In [0]:
df.select(F.count('Quantity'), F.sum('Quantity'), F.avg('Quantity'), F.mean('Quantity')).show()

+---------------+-------------+----------------+----------------+
count(Quantity)|sum(Quantity)| avg(Quantity)| avg(Quantity)|
+---------------+-------------+----------------+----------------+
 541909| 5176450|9.55224954743324|9.55224954743324|
+---------------+-------------+----------------+----------------+

### 분산과 표준편차
- 스파크는 표본표준편차뿐만 아니라 모표준편차방식도 지원
  - 모표준분산, 모표준편차: var_pop / stddev_pop

In [0]:
df.select(F.var_pop('Quantity'), F.var_samp('Quantity'),\
         F.stddev_pop('Quantity'), F.stddev_samp('Quantity')).show()


+------------------+------------------+--------------------+---------------------+
 var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
47559.303646609056|47559.391409298754| 218.08095663447796| 218.08115785023418|
+------------------+------------------+--------------------+---------------------+

### 비대칭도와 첨도
- 비대칭도와 첨도 모두 데이터의 변곡점을 측정하는 방법
  - 비대칭도는 데이터 평균의 비대칭 정도를 측정
  - 첨도는 데이터 끝 부분을 측정

In [0]:
df.select(F.skewness('Quantity'), F.kurtosis('Quantity')).show()

+-------------------+------------------+
 skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
-0.2640755761052562|119768.05495536952|
+-------------------+------------------+

### 공분산과 상관관계

In [0]:
df.select(F.corr('InvoiceNo', 'Quantity'), F.covar_samp('InvoiceNo', 'Quantity'), F.covar_pop('InvoiceNo', 'Quantity')).show()

+-------------------------+-------------------------------+------------------------------+
corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
 4.912186085635685E-4| 1052.7280543902734| 1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+

### 복합 데이터 타입의 집계

In [0]:
df.agg(F.collect_list('Country'), F.collect_set('country')).show()

+---------------------+--------------------+
collect_list(Country)|collect_set(country)|
+---------------------+--------------------+
 [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+

## 그룹화

### 표현식을 이용한 그룹화
- count함수는 select구문에 표현식으로 지정하기보다 agg메서드를 사용하는 것이 좋음
  - agg메서드는 여러 집계 처리를 한 번에 지정할 수 있음

In [0]:
df.groupBy('InvoiceNo').agg(
  F.count('Quantity').alias('quan'),
  F.expr('count(Quantity)')
).show(5)

+---------+----+---------------+
InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
 536596| 6| 6|
 536938| 14| 14|
 537252| 1| 1|
 537691| 20| 20|
 538041| 1| 1|
+---------+----+---------------+
only showing top 5 rows

### 맵을 이용한 그룹화

In [0]:
df.groupBy('InvoiceNo').agg(F.expr('avg(Quantity)'),F.expr('stddev_pop(Quantity)')).show(5)

+---------+------------------+--------------------+
InvoiceNo| avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
 536596| 1.5| 1.1180339887498947|
 536938|33.142857142857146| 20.698023172885524|
 537252| 31.0| 0.0|
 537691| 8.15| 5.597097462078001|
 538041| 30.0| 0.0|
+---------+------------------+--------------------+
only showing top 5 rows